# Import Statements

In [83]:
from scipy.stats import norm, gamma
import time, datetime
import numpy as np, pandas as pd
from os.path import abspath, dirname, join
import pyodbc

import importlib.util
spec = importlib.util.spec_from_file_location("useful_func", r"C:\Users\armando_borjas\Documents\Python\Calculators\general_POE.py")
POE = importlib.util.module_from_spec(spec)
spec.loader.exec_module(POE)

pd.set_option("display.max_columns",500)

# Functions 

In [39]:
def sqlserver_sql(q, server = 'SQL2017', db = 'PLAINS_IRASV6_STAGE'):
    driver = '{SQL Server Native Client 11.0}'
    conn = pyodbc.connect("Driver="+driver+";Server="+server+";Database="+db+";Trusted_Connection=yes;")

    # performing query to database
    df = pd.read_sql_query(q,conn)

    conn.close()
    return df

In [3]:
def get_ili_ranges(line):
    q1 = f"""set nocount on;
            select ll.LineName, ld.code [status], r.* from InlineInspectionRange r
            join StationSeries ss on r.BeginStationSeriesId = ss.Id
            join LineLoop ll on ss.LineLoopId = ll.Id
            join ListDomain ld on r.ILIRStatusDomainId = ld.Id
            where ll.LineName like '%{line}%'
            order by r.ILIRStartDate desc"""

    return sqlserver_sql(q1)

In [46]:
def get_features_for_poe(ILIRID, start=1, amt=999999999):
    query = f"""select
            c.RN,
            ll.linename [line],

            ld4.code [vendor], 

            format(r.ILIRStartDate,'yyyy-MM-dd') [ILIRStartDate],
            ld3.code [tool],

            f.ILIFFeatureNumber [FeatureID],
            ld.code [status], 
            ld2.code [type],
            (f.StationNum*mlv.MultiplierNum+mlv.FactorNum) [chainage_m], 
            f.ILIFSurfaceInd, 
            f.ILIFPeakDepthPct [depth_fraction], 
            f.ILIFLength [length_mm], 
            f.ILIFWidth [width_mm],
            f.GrowthRate [vendor_cgr_mmpyr],
            f.GrowthRateStdDev [vendor_cgr_sd],

            format(a.PipeInserviceDate,'yyyy-MM-dd') [install_date],
            a.PipeOutsideDiameter [OD_inch],
            a.PipeWallThickness [WT_mm],
            a.PipeGrade [grade_MPa],
            a.PipeToughness [toughness_J],
            a.[begin_ps_c],
            a.[end_ps_c],

            b.MAOP_kPa,
            b.begin_maop_c,
            b.end_maop_c  

            from InlineInspectionFeature f

            left join ListDomain ld on f.ILIFStatusDomainId = ld.Id
            left join ListDomain ld2 on f.ILIFTypeDomainId = ld2.Id
            left join StationSeries ss on f.StationSeriesId = ss.id
            left join LineLoop ll on ss.LineLoopId = ll.Id
            left join inlineinspectionrange r on f.inlineinspectionrangeid = r.id
            left join ListDomain ld3 on r.ILIRToolDomainId = ld3.Id
            left join InlineInspection i on f.InlineInspectionId = i.Id
            left join ListDomain ld4 on i.ILICompanyDomainId = ld4.Id
            left join MLVCorrection mlv on f.StationSeriesId = mlv.StationSeriesId

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                ps.EffectiveStartDate,
                ps.PipeInserviceDate,
                ps.PipeOutsideDiameter,
                ps.PipeWallThickness,
                ps.PipeGrade,
                ps.PipeToughness,
                (ps.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum) [begin_ps_c],
                (ps.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum) [end_ps_c]
                from PipeSegment ps
                join StationSeries ss on ps.BeginStationSeriesId = ss.id
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on ps.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on ps.EndStationSeriesId = mlv2.StationSeriesId
                where ps.EffectiveEndDate is null
                ) a on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between a.[begin_ps_c] and a.[end_ps_c]) and a.LinloopId = ll.id

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                maop.EffectiveEndDate,
                maop.BeginStationSeriesId,
                maop.MaxAllowablePressure [MAOP_kPa],
                maop.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum [begin_maop_c],
                maop.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum [end_maop_c]
                from maop maop
                join StationSeries ss on ss.id = maop.BeginStationSeriesId
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on maop.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on maop.EndStationSeriesId = mlv2.StationSeriesId
                where maop.EffectiveEndDate is null
                ) b on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between b.[begin_maop_c] and b.[end_maop_c]) and b.[LinloopId] = ll.id 

            left join
                (select ROW_NUMBER() over(partition by ff.InlineInspectionRangeId order by ff.id asc)  [RN],
                ff.id,
                ff.InlineInspectionRangeId
                from InlineInspectionFeature ff
                )  c on f.Id = c.Id and f.InlineInspectionRangeId = c.InlineInspectionRangeId

            where f.InlineInspectionRangeId = {ILIRID} and (c.RN between {start} and {start+amt})
            order by chainage_m asc"""
    
    return sqlserver_sql(query)

In [5]:
def get_cracks_for_poe(ILIRID, start=1, amt=999999999):
    query = f"""select
            c.RN,
            ll.linename [line],

            ld4.code [vendor], 

            format(r.ILIRStartDate,'yyyy-MM-dd') [ILIRStartDate],
            ld3.code [tool],

            f.ILICAAnomalyNumber [FeatureID],
            ld.code [status], 
            ld2.code [type],
            (f.StationNum*mlv.MultiplierNum+mlv.FactorNum) [chainage_m], 
            f.ILICASurfaceInd, 
            f.ILICADepthPct [depth_fraction], 
            f.ILICALength [length_mm], 
            f.ILICAWidth [width_mm], 

            format(a.PipeInserviceDate,'yyyy-MM-dd') [install_date],
            a.PipeOutsideDiameter [OD_inch],
            a.PipeWallThickness [WT_mm],
            a.PipeGrade [grade_MPa],
            a.PipeToughness [toughness_J],
            a.[begin_ps_c],
            a.[end_ps_c],

            b.MAOP_kPa,
            b.begin_maop_c,
            b.end_maop_c  

            from InlineInspectionCrackAnomaly f

            left join ListDomain ld on f.ILICAStatusDomainId = ld.Id
            left join ListDomain ld2 on f.ILICATypeDomainId = ld2.Id
            left join StationSeries ss on f.StationSeriesId = ss.id
            left join LineLoop ll on ss.LineLoopId = ll.Id
            left join inlineinspectionrange r on f.inlineinspectionrangeid = r.id
            left join ListDomain ld3 on r.ILIRToolDomainId = ld3.Id
            left join InlineInspection i on f.InlineInspectionId = i.Id
            left join ListDomain ld4 on i.ILICompanyDomainId = ld4.Id
            left join MLVCorrection mlv on f.StationSeriesId = mlv.StationSeriesId

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                ps.EffectiveStartDate,
                ps.PipeInserviceDate,
                ps.PipeOutsideDiameter,
                ps.PipeWallThickness,
                ps.PipeGrade,
                ps.PipeToughness,
                (ps.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum) [begin_ps_c],
                (ps.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum) [end_ps_c]
                from PipeSegment ps
                join StationSeries ss on ps.BeginStationSeriesId = ss.id
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on ps.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on ps.EndStationSeriesId = mlv2.StationSeriesId
                where ps.EffectiveEndDate is null
                ) a on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between a.[begin_ps_c] and a.[end_ps_c]) and a.LinloopId = ll.id

            left join 
                (select ll.id [LinloopId],
                ll.LineName,
                maop.EffectiveEndDate,
                maop.BeginStationSeriesId,
                maop.MaxAllowablePressure [MAOP_kPa],
                maop.BeginStationNum*mlv1.MultiplierNum+mlv1.FactorNum [begin_maop_c],
                maop.EndStationNum*mlv2.MultiplierNum+mlv2.FactorNum [end_maop_c]
                from maop maop
                join StationSeries ss on ss.id = maop.BeginStationSeriesId
                join LineLoop ll on ss.LineLoopId = ll.Id
                join MLVCorrection mlv1 on maop.BeginStationSeriesId = mlv1.StationSeriesId
                join MLVCorrection mlv2 on maop.EndStationSeriesId = mlv2.StationSeriesId
                where maop.EffectiveEndDate is null
                ) b on ((f.StationNum*mlv.MultiplierNum+mlv.FactorNum) between b.[begin_maop_c] and b.[end_maop_c]) and b.[LinloopId] = ll.id 

            left join
                (select ROW_NUMBER() over(partition by ff.InlineInspectionRangeId order by ff.id asc)  [RN],
                ff.id,
                ff.InlineInspectionRangeId
                from InlineInspectionCrackAnomaly ff
                )  c on f.Id = c.Id and f.InlineInspectionRangeId = c.InlineInspectionRangeId

            where f.InlineInspectionRangeId = {ILIRID} and (c.RN between {start} and {start+amt})
            order by chainage_m asc"""
    
    return sqlserver_sql(query)

# Analysis

In [44]:
sqlserver_sql("""select ll.linename, f.inlineinspectionrangeid, f.GrowthRate, f.GrowthRateStdDev from inlineinspectionfeature f
                join stationseries ss on f.StationSeriesId = ss.id
                join lineloop ll on ss.lineloopid = ll.id
                where f.GrowthRate is not null""").groupby(['linename','inlineinspectionrangeid']).count()

,,GrowthRate,GrowthRateStdDev
linename,inlineinspectionrangeid,,
CADOTTE LAKE TO UTIKUMA STATION NPS 20,2696.0,2109,0
EMPRESS TO LAPORTE NPS 10,2702.0,1794,0
MANSON TO RAPID CITY NPS 6,2686.0,138,0
PORTAGE TO FORT WHYTE NPS 6,2677.0,119,0
SARNIA CONDENSATE TO DITCH NPS 8,1065.0,242,0
SARNIA TO ST CLAIR TERMINAL NPS 8,2371.0,17983,0
WINDSOR TO SARNIA NPS 12,2339.0,1113,0


In [43]:
sqlserver_sql("""select ll.linename, ld.code, a.count_f, r.* from inlineinspectionrange r
                join listdomain ld on r.ilirtooldomainid = ld.id
                join (select ca.inlineinspectionrangeid, count(ca.id) [count_f] from inlineinspectionfeature ca
                        join listdomain ld on ca.ilifstatusdomainid = ld.id
                        where ld.code = 'Active'
                        group by ca.inlineinspectionrangeid) a on r.id = a.inlineinspectionrangeid
                join stationseries ss on r.beginstationseriesid = ss.id
                join lineloop ll on ss.lineloopid = ll.id
                where ld.code like '%mfl%' or ld.code like '%ultra%'""").sort_values(['linename','ILIRStartDate'], ascending=[True,False])\
.assign(length = lambda x: x.EndStationNum - x.BeginStationNum)[['linename','code','count_f','length','ILIRStartDate','Id']].query('linename.isin(@pipes)')

,linename,code,count_f,length,ILIRStartDate,Id
687,CADOTTE LAKE TO UTIKUMA STATION NPS 20,MFL/Geometry,138731,139083.2620,2020-01-22,2696.0
688,CADOTTE LAKE TO UTIKUMA STATION NPS 20,MFL/Geometry,74839,139083.2620,2016-02-17,2697.0
689,CADOTTE LAKE TO UTIKUMA STATION NPS 20,MFL,25180,139083.2620,2011-01-25,2699.0
690,CADOTTE LAKE TO UTIKUMA STATION NPS 20,MFL/Geometry,19283,139083.2620,2007-04-26,2700.0
625,EMPRESS TO LAPORTE NPS 10,MFL/Geometry,4456,78453.1900,2020-02-20,2702.0
626,EMPRESS TO LAPORTE NPS 10,MFL/Geometry,640,78453.1900,2016-12-14,2703.0
627,EMPRESS TO LAPORTE NPS 10,MFL/Geometry,3621,78453.1900,2016-09-19,2704.0
628,EMPRESS TO LAPORTE NPS 10,MFL,1708,78453.1900,2009-12-11,2705.0
652,MANSON TO RAPID CITY NPS 6,SpirAll MFL/Geometry,1882,106971.6820,2020-02-12,2686.0
653,MANSON TO RAPID CITY NPS 6,MFL,1086,106971.6820,2016-03-01,2690.0


In [72]:
features = get_features_for_poe(1065)

In [73]:
features.describe(include='all')

,RN,line,vendor,ILIRStartDate,tool,FeatureID,status,type,chainage_m,ILIFSurfaceInd,depth_fraction,length_mm,width_mm,vendor_cgr_mmpyr,vendor_cgr_sd,install_date,OD_inch,WT_mm,grade_MPa,toughness_J,begin_ps_c,end_ps_c,MAOP_kPa,begin_maop_c,end_maop_c
count,5283.000000,5283,5283,5283,5283,5283,5283,5283,5283.000000,5283,5283.000000,5283.000000,5283.000000,242.000000,0,5283,5283.000,5283.000000,5283.0,5279.0,5283.000000,5283.000000,5283.000000,5283.000000,5283.000000
unique,NaN,1,1,1,1,5283,2,3,NaN,1,NaN,NaN,NaN,NaN,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,5987,Active,Metal Loss,NaN,E,NaN,NaN,NaN,NaN,NaN,1969-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,5283,5283,5283,5283,1,5282,5268,NaN,5283,NaN,NaN,NaN,NaN,NaN,5279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,2642.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3322.343406,NaN,0.142059,22.314215,23.996782,0.037774,NaN,NaN,8.625,4.829292,290.0,20.0,2658.389365,4048.606893,9098.330494,5.907457,7802.508556
std,1525.215067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2322.528481,NaN,0.028159,53.985116,11.528071,0.094365,NaN,NaN,0.000,0.414872,0.0,0.0,1866.129649,2598.055727,60.656161,214.628564,8.521847
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.580000,NaN,0.110000,4.000000,11.000000,0.000000,NaN,NaN,8.625,4.780000,290.0,20.0,5.198000,31.158000,6895.000000,0.000000,7802.274000
25%,1321.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,811.252500,NaN,0.120000,14.000000,18.000000,0.000000,NaN,NaN,8.625,4.780000,290.0,20.0,791.744000,1210.944000,9100.000000,0.000000,7802.274000
50%,2642.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3309.328000,NaN,0.130000,18.000000,23.000000,0.000000,NaN,NaN,8.625,4.780000,290.0,20.0,2940.774000,4670.238000,9100.000000,0.000000,7802.274000
75%,3962.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5350.731000,NaN,0.150000,26.000000,27.000000,0.000000,NaN,NaN,8.625,4.780000,290.0,20.0,4711.152000,7158.458000,9100.000000,0.000000,7802.274000


In [74]:
features

,RN,line,vendor,ILIRStartDate,tool,FeatureID,status,type,chainage_m,ILIFSurfaceInd,depth_fraction,length_mm,width_mm,vendor_cgr_mmpyr,vendor_cgr_sd,install_date,OD_inch,WT_mm,grade_MPa,toughness_J,begin_ps_c,end_ps_c,MAOP_kPa,begin_maop_c,end_maop_c
0,5283,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,6,Active,Metal Loss,5.580,E,0.20,18.0,29.0,0.344348,None,1969-01-01,8.625,7.92,290.0,20.0,5.198,31.158,9100.0,0.000,7802.274
1,5282,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,7,Active,Metal Loss,19.750,E,0.22,80.0,37.0,0.172174,None,1969-01-01,8.625,7.92,290.0,20.0,5.198,31.158,9100.0,0.000,7802.274
2,5281,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,8,Active,Metal Loss,23.728,E,0.12,14.0,14.0,NaN,None,1969-01-01,8.625,7.92,290.0,20.0,5.198,31.158,9100.0,0.000,7802.274
3,5280,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,11,Active,Metal Loss,34.644,E,0.12,18.0,18.0,NaN,None,1969-01-01,8.625,4.78,290.0,20.0,31.158,444.738,9100.0,0.000,7802.274
4,5279,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,14,Active,Metal Loss,36.686,E,0.17,58.0,69.0,NaN,None,1969-01-01,8.625,4.78,290.0,20.0,31.158,444.738,9100.0,0.000,7802.274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5278,5,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,6780,Active,Metal Loss,7793.350,E,0.18,22.0,29.0,NaN,None,1969-01-01,8.625,9.53,290.0,20.0,7764.648,7802.274,9100.0,0.000,7802.274
5279,4,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,6797,Active,Metal Loss,7839.550,E,0.15,10.0,14.0,NaN,None,1999-01-01,8.625,9.53,290.0,NaN,7814.433,8112.064,6895.0,7802.274,8112.064
5280,3,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,6798,Active,Metal Loss,7840.450,E,0.15,18.0,14.0,NaN,None,1999-01-01,8.625,9.53,290.0,NaN,7814.433,8112.064,6895.0,7802.274,8112.064
5281,2,SARNIA CONDENSATE TO DITCH NPS 8,Onstream,2016-02-19,MFL/Geometry,6799,Active,Metal Loss,7840.494,E,0.13,14.0,14.0,NaN,None,1999-01-01,8.625,9.53,290.0,NaN,7814.433,8112.064,6895.0,7802.274,8112.064


In [75]:
features.query("status == 'Active'",inplace=True)

In [76]:
features.query("~vendor_cgr_mmpyr.isnull()",inplace=True)

In [85]:
def cgr(**kwargs):
#     alpha = kwargs['vcgr']/kwargs['vcgr_sd']
#     return np.where(alpha > 3.0,
#                     np.maximum(0, norm.ppf(kwargs['random'], loc=kwargs['vcgr'], scale=kwargs['vcgr_sd']))/25.4,
#                     np.where(kwargs['surface'] == 'E', POE.cgr_weibull(kwargs['random'], 1.6073, 0.1) / 25.4,
#                                 POE.cgr_weibull(kwargs['random'], 3.2962, 0.1) / 25.4))
    return np.where(kwargs['surface'] == 'E',
                POE.cgr_weibull(kwargs['random'], 1.6073, 0.1) / 25.4,
                POE.cgr_weibull(kwargs['random'], 3.2962, 0.1) / 25.4)
    # return 12.5/1000.
#     return kwargs['depth']/kwargs['ILI_age']

config = dict(iterations=1_000_000,cgr=cgr)

corr = POE.MonteCarlo('CORR', config=config)
corr.df = features.iloc[0:40,:].copy()
# scc.df.select_dtypes(np.float) =  scc.df.select_dtypes(np.float).astype(np.float32)
corr.process_dates()
corr.run()#split_iterations=True,iterations_split=10)
corr.result

Model: CORR POE Simulation
Count of anomalies: 40
Iterations: 1,000,000
Date of analysis: 2020-07-13
Leak threshold modifier: 0.8
Rupture threshold modifier: 1.0
Aggregated POE for these features is 0.00025597691807321965.

Calculation took 156.1075 seconds.


,FeatureID,fail_count,iterations,rupture_count,leak_count,nan,PDP_frac,flength,POE,POE_l,POE_r,1-POE
0,6,0,1000000,0,0,0.0,0.20,18.0,0.000000,0.000000,0.000000,1.000000
1,7,0,1000000,0,0,0.0,0.22,80.0,0.000000,0.000000,0.000000,1.000000
2,20,0,1000000,0,0,0.0,0.19,14.0,0.000000,0.000000,0.000000,1.000000
3,21,1,1000000,0,1,0.0,0.21,28.0,0.000001,0.000001,0.000000,0.999999
4,22,1,1000000,0,1,0.0,0.21,14.0,0.000001,0.000001,0.000000,0.999999
5,28,0,1000000,0,0,0.0,0.18,8.0,0.000000,0.000000,0.000000,1.000000
6,31,1,1000000,0,1,0.0,0.16,26.0,0.000001,0.000001,0.000000,0.999999
7,37,0,1000000,0,0,0.0,0.19,10.0,0.000000,0.000000,0.000000,1.000000
8,72,0,1000000,0,0,0.0,0.20,10.0,0.000000,0.000000,0.000000,1.000000
9,73,0,1000000,0,0,0.0,0.20,10.0,0.000000,0.000000,0.000000,1.000000


In [86]:
corr.merge_result('FeatureID').sort_values('chainage_m').to_clipboard()#.style.format({'POE':'{:.4e}'})

In [63]:
sqlserver_sql('''select * from virtualtable vt''').query('VirtualTableName.str.contains("SCC")')

,Id,VirtualTableName,VirtualTableOverlapFlg,VirtualTableComment,LastModByUserId,LastModDateTime
0,1.0,SCCPs,0.0,DO NOT USE,DYNAMIC_RISK\Aravind_Peddaboina,2018-06-14 09:05:23.660
26,27.0,SCCDADate,0.0,SCC DA Date,DYNAMIC_RISK\martin_friese,2009-06-09 09:03:24.143
27,28.0,SCCHistory,0.0,SCC History,DYNAMIC_RISK\martin_friese,2009-06-09 09:03:24.143
60,75.0,SCCReassessmentInterval,0.0,SCC Reassessment Interval,,2012-06-27 08:09:59.000
87,102.0,SCCPOI,0.0,DO NOT USE,DYNAMIC_RISK\Aravind_Peddaboina,2018-06-14 09:05:23.660
88,103.0,SCCPOD,0.0,DO NOT USE,DYNAMIC_RISK\Aravind_Peddaboina,2018-06-14 09:05:23.660
89,104.0,SCCCrackGrowthRate,0.0,SCC Crack Growth Rate (mm/yr),DYNAMIC_RISK\winston_lee,2015-07-07 07:42:45.667
202,222.0,SCCNNHistoryStatus,0.0,SCC History Status,DYNAMIC_RISK\Aravind_Peddaboina,2018-06-18 09:43:39.020


In [ ]:
sqlserver_sql('''select * from virtualtabledatanum vtn
                join virtualtablestationing vts on vtn.virtualtablestationingid = vts.virtualtablestationingid and vts.virtualtableid = vtn.virtualtableid''')